In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy, log_loss
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset
from pipeline.image_extracting import extract_faces, FastMTCNN

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#%%script false

fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.6,
    keep_all=True,
    device=device
)

Couldn't find program: 'false'


In [3]:
y_train = pd.read_csv('data\\frames\\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(94949, 18265)

In [17]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((299, 299)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

In [4]:
%%script false

batch_size = 10
normalize = torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))#torchvision.transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

x_train = load_img_dataset('data/frames/', batch_size, resize=256, normalize=normalize)
x_val = load_img_dataset('data/validation_set/', batch_size, resize=256, normalize=normalize)

In [5]:
%%script false

model = models.resnet18(pretrained=True)

#for param in model.parameters():
#    model.requires_grad = False

in_features = model.fc.in_features
model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 2)
        )

Couldn't find program: 'false'


In [6]:
%%script false
model.load_state_dict(torch.load(r'best.pkl'))

if isinstance(model, torch.nn.DataParallel):
    model = model.module

Couldn't find program: 'false'


In [7]:
%%script false

model_path = r'faceforensics++_models_subset\face_detection\xception\all_c40.p'
model = torch.load(model_path)
torch.save(model.state_dict(), "all_c40.pth")

Couldn't find program: 'false'


In [8]:
#model, *_ = model_selection(modelname='xception', num_out_classes=2, dropout=None)
model = MesoInception4()
loss =  torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [9]:
#model.load_state_dict(torch.load("pretrained/xception/all_c40.pth"))
#model.eval()

In [10]:
model = model.to(device)

In [11]:
%%script false

validate_img(model, x_val, y_train, loss, accuracy, device, batch_size, print_results=True, inference=nn.Softmax(dim=1))

Couldn't find program: 'false'


In [12]:
# all_c23 | Validation: metrics  0.55177665 loss  0.7217166
# all_c40 | Validation: metrics  0.56040615 loss  0.72333634
# all_raw | Validation: metrics  0.4967005 loss  0.75162345

# full_c23 | Validation: metrics  0.45558375 loss  0.7241818
# full_c40 | Validation: metrics  0.49416244 loss  0.7669457
# full_raw | Validation: metrics  0.5340101 loss  0.6878851

# FF++ xception dp=0.5 adam lr=1e-3 wd=0 step_size=5 gamma=0.5 epochs=13 | Validation: metrics  0.56167513 loss  0.7042332 Train: loss 0.691612023774213

In [14]:
%%script false

metric, lost = train_img(model, loss, optimizer, scheduler, x_train, x_val, y_train, accuracy, device, epochs=25, 
                       batch_size=batch_size, 
                       del_net=False, 
                       useInference=False, 
                       inference=nn.Softmax(dim=1), 
                       useScheduler=True,
                       checkpoint=0.1
                      )

Unable to make an epoch: 6 found in y_train. 6 batch size.




Validation: metrics  0.5043147 loss  0.6932251
Train: metrics  0.5390037 loss  0.6906890123901506


Unable to make an epoch: 6 found in y_train. 6 batch size.




Validation: metrics  0.5043147 loss  0.6933624
Train: metrics  0.53876066 loss  0.6906601573688002


Unable to make an epoch: 6 found in y_train. 6 batch size.



...

Unable to make an epoch: 6 found in y_train. 6 batch size.




Validation: metrics  0.5043147 loss  0.6934868
Train: metrics  0.5390037 loss  0.6902998751018056


Unable to make an epoch: 6 found in y_train. 6 batch size.




Validation: metrics  0.5007614 loss  0.6933024
Train: metrics  0.5393682 loss  0.6901122494572423


KeyboardInterrupt: 

In [ ]:
from operator import itemgetter
min_lost = min(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = min(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
'''model = torch.load(PATH)
model = model.to(device)
model.eval()'''

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, print_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_graphic=True, inference=nn.Softmax(dim=1))

In [ ]:
#torch.save(model, PATH)